# Extract MODIS Site Data and Generate Samples
## MODIS Site Data
For each site in the LFMC sample data, extract the full time series of MODIS reflectance and snow-cover data, and save to CSV files. Note: if the output csv files already exist they are assumed to be correct and are not over-written. The data is gap-filled after being saved but before being used to create the MODIS sample data.

## MODIS Sample Data
For each sample, extract the timeseries MODIS reflectance data. The timeseries length is determined by the MODIS_TS_LENGTH value. The sample is rejected if the full timeseries cannot be extracted (start/end outside the full site time series). The extracted MODIS data is combined to a single dataframe and saved. Another LFMC sample dataset containing only the valid samples is also created.

## Input Files
- `LFMC_sites.csv` and `samples_365days.csv` created by the `Extract Auxiliary Data.ipynb` and `Extract ERA5 DATA.ipynb` notebooks.

## Output Files
- The extracted sites data are created in `MODIS_DIR` (by default `MCD43A4` located in `LFMC_DATA_DIR/GEE_DIR`). The directory will contain a CSV file for each site.
- The extracted MODIS data for each samples and updated samples data are created in `DATASETS_DIR`. File names include the time series length (i.e. 730days) of the extracted MODIS data. So, with the default settings they are `modis_730days.csv` and `samples_730days.csv`.

## Notes
1. This notebook should be run after running the `Extract Auxiliary Data.ipynb` and `Extract ERA5 DATA.ipynb` notebooks.
2. The generated name for `GEE_DIR` includes the projection and scale of the extracted GEE data. 
3. It will take about 8.5 hours to run if there are no existing site extracts.
4. Data for 67724 samples will be extracted. If the samples input is the ERA5 extract sample output, there should be no invalid or discarded samples.
5. There should be no invalid sites, but occasionally extraction from GEE will fail for a site. If this happens re-run the notebook (keep the existing site CSV files so they are not re-extracted).

In [1]:
import os
os.environ["HDF5_DISABLE_VERSION_CHECK"] = "1"
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime
from datetime import timedelta

import initialise
import common
from data_extract_utils import extract_timeseries_data, sort_key
from timeseries_extractor import GeeTimeseriesExtractor

### Program parameters and constants

GEE Parameters
- Reflectance product is MCD43A4 - daily reflectance using 8-day composites
- Scale set to use native MODIS resolution
- Extracts two years' of data for each sample to allow for a 1-year time series with a 1-year lead time

In [2]:
START_DATE = "2000-03-01"
END_DATE = "2019-01-01"  # Final day retrieved will be 2018-12-31

# MODIS time series constants
MODIS_TS_LENGTH = 365
MODIS_TS_OFFSET = 1
MODIS_TS_FREQ = 1

# MODIS data details
PRODUCT = "MODIS/006/MCD43A4"
BANDS = ["Nadir_Reflectance_Band1",
         "Nadir_Reflectance_Band2",
         "Nadir_Reflectance_Band3",
         "Nadir_Reflectance_Band4",
         "Nadir_Reflectance_Band5",
         "Nadir_Reflectance_Band6",
         "Nadir_Reflectance_Band7"]
# Extract data using default MODIS scale and projection
SCALE = 463.3127
PROJ = "SR-ORG:6974"

EARLIEST_SAMPLE = datetime.strptime(START_DATE, '%Y-%m-%d') + timedelta(
    days=MODIS_TS_LENGTH * MODIS_TS_FREQ + MODIS_TS_OFFSET  - 1)

### Directories and Files

In [3]:
# Sub-directories for GEE extracts
GEE_DIR = f"GEE_{common.PROJ.replace(':', '-')}_{int(common.SCALE)}"
MODIS_DIR = os.path.join(common.LFMC_DATA_DIR, GEE_DIR, "MCD43A4")

# File Names
SITES_INPUT = os.path.join(common.DATASETS_DIR, "LFMC_europe_sites.csv")
SAMPLES_INPUT = os.path.join(common.DATASETS_DIR, f"europe_samples_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")
SAMPLES_OUTPUT = os.path.join(common.DATASETS_DIR, f"europe_samples_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days_new.csv")
MODIS_OUTPUT = os.path.join(common.DATASETS_DIR, f"europe_modis_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")

# Create output directories if necessary
if not os.path.exists(MODIS_DIR):
    os.makedirs(MODIS_DIR)

## Main Processing
Connect to GEE

In [4]:
import ee
ee.Initialize()

### Generate MODIS sample data

For each site, get the sample data for each sample at the site

Note: This gets the data using the default MODIS scale and projection. To change the scale and/or projection, add calls to the `GeeTimeseriesExtractor.setProjScale` method

In [6]:
sites = pd.read_csv(SITES_INPUT, float_precision="high")
samples = pd.read_csv(SAMPLES_INPUT, float_precision="high")
modis_extractor = GeeTimeseriesExtractor(PRODUCT, BANDS, START_DATE, END_DATE,
                                         dir_name=MODIS_DIR)
modis_data, valid_data, invalid_pixels, invalid_sites = extract_timeseries_data(
    modis_extractor, sites, samples, EARLIEST_SAMPLE,
    MODIS_TS_OFFSET, MODIS_TS_LENGTH, MODIS_TS_FREQ)

Processing site C2_1
Extracting data for C2_1 (lat: 38.92292 long: -1.71649)
Processing site C2_2
Extracting data for C2_2 (lat: 38.30625 long: -2.15313)
Processing site C2_3
Extracting data for C2_3 (lat: 38.20625 long: -2.30925)
Processing site C2_4
Extracting data for C2_4 (lat: 36.75625 long: -5.43723)
Processing site C2_5
Extracting data for C2_5 (lat: 36.76042 long: -5.38552)
Processing site C2_6
Extracting data for C2_6 (lat: 42.29375 long: -0.90971)
Processing site C2_7
Extracting data for C2_7 (lat: 42.28542 long: -0.89833)
Processing site C2_8
Extracting data for C2_8 (lat: 42.27708 long: -1.01647)
Processing site C2_9
Extracting data for C2_9 (lat: 42.40208 long: -0.92821)
Processing site C2_10
Extracting data for C2_10 (lat: 41.25208 long: -1.2165)
Processing site C2_11
Extracting data for C2_11 (lat: 41.19792 long: -1.18227)
Processing site C2_12
Extracting data for C2_12 (lat: 41.25625 long: -1.29971)
Processing site C2_13
Extracting data for C2_13 (lat: 41.31042 long: -1

Summary of sites/samples not extracted

In [7]:
print(f'Invalid sites: {len(invalid_sites)}; Invalid pixels: {len(invalid_pixels)}; ')
print(invalid_sites)
print(invalid_pixels)

Invalid sites: 0; Invalid pixels: 0; 
[]
[]


### Save Results
Save and display sample reflectance data

In [8]:
modis_data = pd.DataFrame(modis_data)
ts_days = (MODIS_TS_LENGTH - 1) * MODIS_TS_FREQ
modis_data.columns = ["ID"] + [f'{day-MODIS_TS_OFFSET:04}_{band+1}'
                               for day in range(-ts_days, 1, MODIS_TS_FREQ)
                               for band in range(len(BANDS))]
modis_data.sort_values('ID', inplace=True, key=lambda x: x.apply(sort_key))
modis_data.to_csv(MODIS_OUTPUT, index=False)
modis_data

,ID,-365_1,-365_2,-365_3,-365_4,-365_5,-365_6,-365_7,-364_1,-364_2,...,-002_5,-002_6,-002_7,-001_1,-001_2,-001_3,-001_4,-001_5,-001_6,-001_7
0,C2_1_1,1184,2401,483,941,2887,2830,2045,1188,2407,...,2916,2482,1757,1125,2720,544,962,2919,2484,1759
1,C2_1_3,1588,2562,749,1237,3339,3265,2486,1548,2534,...,3300,3150,2384,1512,2573,725,1204,3289,3126,2367
2,C2_2_1,725,1981,379,683,2348,1991,1277,728,1990,...,2472,2060,1349,722,1937,385,704,2487,2050,1349
3,C2_2_10,941,2146,492,833,2617,2488,1599,940,2145,...,2763,2588,1683,925,2183,483,845,2759,2593,1681
4,C2_3_1,829,1987,407,718,2693,2580,1791,823,1975,...,2633,2489,1751,851,2098,424,745,2692,2485,1802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8393,C15_1_13,350,3044,164,512,2912,1556,594,353,3010,...,2999,1691,650,341,3089,172,471,2989,1693,660
8394,C15_1_19,334,2924,162,470,2881,1515,576,333,2922,...,2798,1659,654,395,2844,189,482,2789,1656,651
8395,C15_1_25,352,3006,169,479,2797,1604,623,352,3013,...,2759,1600,635,354,2837,163,453,2757,1597,635
8396,C15_1_31,342,2720,179,460,2556,1429,538,332,2720,...,2474,1447,498,358,2548,179,447,2455,1391,482


Save and display the valid samples

In [9]:
# valid_samples = samples[valid_data].sort_values('ID', key=lambda x: x.apply(sort_key))
# valid_samples.to_csv(SAMPLES_OUTPUT, index=False)
# valid_samples